In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

train = pd.read_csv('/content/drive/MyDrive/인사이콘 2/데이터전처리데이터_train.csv', index_col=0)
test = pd.read_csv('/content/drive/MyDrive/인사이콘 2/데이터전처리데이터_test.csv', index_col=0)
submission = pd.read_csv('/content/drive/MyDrive/인사이콘 2/submission_sample.csv')
real

In [ ]:
test.head()

,gender,car,reality,income_type,edu_type,family_type,house_type,DAYS_BIRTH,work_phone,phone,...,occyp_type,begin_month,ability,before_EMPLOYED,DAYS_EMPLOYED_log,income_total_dev,income_total_log,familiy+child,FAM_CHILD_SUM,INCOME/FAMILY(RATIO)
0,0,1,0,0,1,1,1,5,0,0,...,4,16.0,-7.097792,-13372,2.348893,20.173004,2.734368,2.0,2.0,7.200
1,0,1,0,4,4,0,1,6,1,1,...,16,46.0,-7.459152,-15801,2.113777,9.868625,2.818398,2.0,2.0,7.875
2,0,0,1,2,4,1,1,7,0,1,...,3,12.0,-4.505406,-11120,3.288913,29.067590,2.674149,2.0,2.0,6.750
3,1,1,0,0,4,1,1,2,0,0,...,9,10.0,-14.867186,-9348,2.188971,13.021729,3.157000,4.0,4.0,7.500
4,0,0,1,2,4,3,1,5,0,0,...,12,8.0,-16.235719,-13251,2.522765,262.717523,3.586293,1.0,1.0,35.100


In [ ]:
X = train.drop(['familiy+child', 'credit'], axis=1)
Y = train['credit']

In [ ]:
train.columns

Index(['gender', 'car', 'reality', 'income_type', 'edu_type', 'family_type',
       'house_type', 'DAYS_BIRTH', 'work_phone', 'phone', 'email',
       'occyp_type', 'begin_month', 'credit', 'ability', 'before_EMPLOYED',
       'DAYS_EMPLOYED_log', 'income_total_dev', 'income_total_log',
       'familiy+child', 'FAM_CHILD_SUM', 'INCOME/FAMILY(RATIO)'],
      dtype='object')

In [ ]:
!pip install wandb -qqq
import wandb
wandb.login()

     |████████████████████████████████| 1.8 MB 5.5 MB/s 
     |████████████████████████████████| 181 kB 70.1 MB/s 
     |████████████████████████████████| 145 kB 71.8 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
sweep_config = {
    "method": "random", # try grid or random
    "metric": {
      "name": "accuracy",
      "goal": "maximize"   
    },
    "parameters": {
        "objective": {
            "values": ['multi:softmax']
        },
        "n_estimator": {
            "values": [50, 75, 100, 200, 300, 500, 1000]
        },
        "max_depth": {
            "values": [16, 18, 20, 22, 24, 26]
        },
        "learning_rate": {
            "values": [0.03, 0.05, 0.07, 0.1]
        },
        "subsample": {
            "values": [1, 0.5, 0.3]
        },
        "gamma": {
            "values": [0, 1, 2, 3]
        },
        "min_child_weight":{
            "values": [1, 3, 5]
        },
        "colsample_bytree":{
            "values": [0.5, 0.75, 1]
        },
        "colsample_bylevel":{
            "values": [0.5, 0.75, 1]
        },
        "lambda":{
            "values": [0.5, 1, 1.5, 2]
        }

    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="XGBoost-sweeps")

Create sweep with ID: 13s1sszj
Sweep URL: https://wandb.ai/perforated_line/XGBoost-sweeps/sweeps/13s1sszj


In [ ]:
# XGBoost model for Pima Indians dataset
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# load data
def train():
  config_defaults = {
    "booster": "gbtree",
    "max_depth": 3,
    "learning_rate": 0.1,
    "subsample": 1,
    "seed": 117,
    "test_size": 0.33,
  }

  wandb.init(config=config_defaults)  # defaults are over-ridden during the sweep
  config = wandb.config

  # load data and split into predictors and targets
  #dataset = loadtxt("pima-indians-diabetes.data.csv", delimiter=",")
  #X, Y = dataset[:, :8], dataset[:, 8]

  # split data into train and test sets
  X_train, X_test, y_train, y_test = train_test_split(X, Y,
                                                      test_size=config.test_size,
                                                      random_state=config.seed)

  # fit model on train
  model = XGBClassifier(booster=config.booster, max_depth=config.max_depth,
                        learning_rate=config.learning_rate, subsample=config.subsample)
  model.fit(X_train, y_train)

  # make predictions on test
  y_pred = model.predict(X_test)
  predictions = [round(value) for value in y_pred]

  # evaluate predictions
  accuracy = accuracy_score(y_test, predictions)
  print(f"Accuracy: {int(accuracy * 100.)}%")
  wandb.log({"accuracy": accuracy})

In [ ]:
wandb.agent(sweep_id, train, count=50)

wandb: Agent Starting Run: 5yk3fxlq with config:
wandb: 	colsample_bylevel: 0.5
wandb: 	colsample_bytree: 0.5
wandb: 	gamma: 3
wandb: 	lambda: 0.5
wandb: 	learning_rate: 0.05
wandb: 	max_depth: 22
wandb: 	min_child_weight: 1
wandb: 	n_estimator: 200
wandb: 	objective: multi:softmax
wandb: 	subsample: 0.3
wandb: Currently logged in as: naian (perforated_line). Use `wandb login --relogin` to force relogin


Accuracy: 71%


accuracy,▁
accuracy,0.71339


wandb: Agent Starting Run: b90vee27 with config:
wandb: 	colsample_bylevel: 0.75
wandb: 	colsample_bytree: 1
wandb: 	gamma: 2
wandb: 	lambda: 2
wandb: 	learning_rate: 0.05
wandb: 	max_depth: 20
wandb: 	min_child_weight: 5
wandb: 	n_estimator: 100
wandb: 	objective: multi:softmax
wandb: 	subsample: 0.5


Accuracy: 70%


...

Accuracy: 71%


accuracy,▁
accuracy,0.71052


wandb: Agent Starting Run: v4lnj9zv with config:
wandb: 	colsample_bylevel: 0.75
wandb: 	colsample_bytree: 0.5
wandb: 	gamma: 3
wandb: 	lambda: 0.5
wandb: 	learning_rate: 0.1
wandb: 	max_depth: 16
wandb: 	min_child_weight: 5
wandb: 	n_estimator: 50
wandb: 	objective: multi:softmax
wandb: 	subsample: 0.3


Accuracy: 70%


accuracy,▁
accuracy,0.70594


In [ ]:
sweep_config = {
    "method": "random", # try grid or random
    "metric": {
      "name": "accuracy",
      "goal": "maximize"   
    },
    "parameters": {
        "learning_rate": {
            "values": [0.05, 0.07]
        },
        "subsample": {
            "values": [0.7, 0.5, 0.3, 0.2]
        },
        "gamma": {
            "values": [0, 0.5, 1, 2]
        },
        "colsample_bytree":{
            "values": [0.5, 0.75, 1]
        },
        "lambda":{
            "values": [0.5, 1, 1.5, 2]
        },
        "sclae_pos_weight":{
            "values": [0.15, 0.3, 0.5, 0.7]
        }

    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="XGBoost-sweeps")

Create sweep with ID: hgrv5qk3
Sweep URL: https://wandb.ai/perforated_line/XGBoost-sweeps/sweeps/hgrv5qk3


In [ ]:
# XGBoost model for Pima Indians dataset
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# load data
def train():
  config_defaults = {
    "n_estimator": 200,
    "objective": 'multi:softmax',
    "booster": "gbtree",
    "max_depth": 16,
    "learning_rate": 0.1,
    "subsample": 1,
    "seed": 117,
    "test_size": 0.33,
  }

  wandb.init(config=config_defaults)  # defaults are over-ridden during the sweep
  config = wandb.config

  # load data and split into predictors and targets
  #dataset = loadtxt("pima-indians-diabetes.data.csv", delimiter=",")
  #X, Y = dataset[:, :8], dataset[:, 8]

  # split data into train and test sets
  X_train, X_test, y_train, y_test = train_test_split(X, Y,
                                                      test_size=config.test_size,
                                                      random_state=config.seed)

  # fit model on train
  model = XGBClassifier(booster=config.booster, max_depth=config.max_depth,
                        learning_rate=config.learning_rate, subsample=config.subsample)
  model.fit(X_train, y_train)

  # make predictions on test
  y_pred = model.predict(X_test)
  predictions = [round(value) for value in y_pred]

  # evaluate predictions
  accuracy = accuracy_score(y_test, predictions)
  print(f"Accuracy: {int(accuracy * 100.)}%")
  wandb.log({"accuracy": accuracy})

In [ ]:
wandb.agent(sweep_id, train, count=50)

wandb: Agent Starting Run: 5gex8bpl with config:
wandb: 	colsample_bytree: 0.75
wandb: 	gamma: 0.5
wandb: 	lambda: 0.5
wandb: 	learning_rate: 0.07
wandb: 	sclae_pos_weight: 0.15
wandb: 	subsample: 0.2
wandb: Currently logged in as: naian (perforated_line). Use `wandb login --relogin` to force relogin


Accuracy: 71%


accuracy,▁
accuracy,0.71052


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: io33iini with config:
wandb: 	colsample_bytree: 1
wandb: 	gamma: 0
wandb: 	lambda: 0.5
wandb: 	learning_rate: 0.07
wandb: 	sclae_pos_weight: 0.3
wandb: 	subsample: 0.7


Accuracy: 70%


...

Accuracy: 70%


accuracy,▁
accuracy,0.70694


wandb: Agent Starting Run: jvmiepta with config:
wandb: 	colsample_bytree: 0.5
wandb: 	gamma: 0.5
wandb: 	lambda: 0.5
wandb: 	learning_rate: 0.05
wandb: 	sclae_pos_weight: 0.15
wandb: 	subsample: 0.5


Accuracy: 70%


accuracy,▁
accuracy,0.70938
